In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"]= os.getenv("HUGGINGFACE_API_TOKEN")

In [5]:
# Read pdf from folders

loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, wh

In [6]:
# Embeddings using Huggingface
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name= "BAAI/bge-small-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings': True}
)

C:\Users\sahil\AppData\Local\Temp\ipykernel_29288\1177215172.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(
c:\Users\sahil\OneDrive\Desktop\Projects\AI Projects\Langchain Project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-0.07903482 -0.01134113 -0.02312095  0.02844462  0.05053345  0.05317827
 -0.01907792  0.03456027 -0.10211367 -0.02915701  0.08524261  0.05650727
 -0.02545437 -0.0330849  -0.00635737  0.04090865 -0.00628106  0.00356743
 -0.03854126  0.03667682 -0.04289803  0.03425248 -0.03116897 -0.03793732
  0.01728392  0.01214926  0.0065312   0.01463566 -0.05529055 -0.15320709
  0.00730848  0.03202945 -0.04701129 -0.01595975  0.01874448  0.02642939
 -0.02306378  0.0843804   0.04182491  0.05278175 -0.03057606  0.01564261
 -0.01689075  0.00529407 -0.02417431  0.00412995 -0.01889938 -0.00150626
 -0.00836939 -0.03390063  0.03515958 -0.00553127  0.04910937  0.05971859
  0.0561596  -0.05105156  0.01475134 -0.0184996  -0.03284643  0.03576627
  0.04947709 -0.00938881 -0.26202112  0.09750332  0.01715691  0.04781387
 -0.00556316 -0.00298307 -0.02207358 -0.04463668 -0.05760488  0.04815875
 -0.0552221   0.01635336  0.03299245  0.0214708   0.01296216  0.01462311
  0.02174953 -0.00202996  0.02099537  0.03353844 -0

In [8]:
# Vectorstore creation 
# Generates and store embeddings for document text using the provided model
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

In [9]:
# Query using similarity search
query = "What is health insurance coverage?"
relevant_docs = vectorstore.similarity_search(query)

print(relevant_docs[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
# Create retriever object 
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})  # {"k":3} =  search for top 3 similar documents
retriever 

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E098D48C20>, search_kwargs={'k': 3})

In [ ]:
# Initialize LLM
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2:1b")
llm

Ollama(model='llama3.2:1b')

In [ ]:
#Create Prompt
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",    # All retrieved documents are concatenated and inserted into the prompt.
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [16]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

The differences in the uninsured rate by state in 2022 are as follows:

- Massachusetts had an increase of 6.1% from 8.0% to 14.1%
- Texas had a decrease of 4.3% from 12.7% to 9.4%
- Maine had an increase of 1.1% from 5.7% to 6.8%

Additionally, Medicaid coverage accounted for a portion of the difference in uninsured rates between states that expanded and those that did not, with percentages as follows:

- Massachusetts: 22.7%
- District of Columbia/Massachusetts: 10.0%
- Non-expansion states: 18.0%
